# The Task

Preprocessing survey data for focadata 

In [310]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date 

1. The data science pipeline expects that NULL values are coded in a consistent fashion. Transform
    all missing values such that they are consistent with one another.
2. De-duplicate the data such that every row is unique. - DONE
3. Convert the year of birth column into an “age” column (i.e. year_of_birth: 1990 => age: 30) - DONE
4. Using the lookup table (postcode_lookup.csv) convert the postcode column into a
    Westminster Constituency Code. - DONE
5. You will see that certain questions are follow-ups, and only get asked according to the response
    to the previous question (“If you answered yes, to the previous question - would you ever ...?”). In
    these cases please combine the two such that the “no” response (in this case) appears in the
    follow-up question, rather than the follow-up column containing missing values for those people
    to whom the question was not presented.
6. Please remove any columns that you think look like junk - DONE (technically)

Bad respondants:
1. Have really fast response times (the survey should take between 8 and 10 minutes)
2. Have repeated answers (e.g. rank all options equally, say they feel the same way about every politician etc)
3. Didn't answer a lot of the questions asked
4. Have odd postcodes/ages

removing nulls 
 - Average all data points and full nans with average/most common 
 - Label nan points as unknown
 - Estimate value from other sources

# Constants

In [268]:
raw_data_fname = r'C:\Users\bless\OneDrive\Documents\focaldatajuniorapplieddatascientist\raw_survey.csv'
postcode_lookup_fname = r'C:\Users\bless\OneDrive\Documents\focaldatajuniorapplieddatascientist\postcode_lookup.csv'

## Loading the data

In [269]:
raw_df = pd.read_csv(raw_data_fname)
postcode_lookup = pd.read_csv(postcode_lookup_fname)

In [270]:
for i,col in enumerate(raw_df.columns):
    print(i,col)

0 is_preview
1 survey_id
2 respondent_id
3 start_time
4 end_time
5 Q1 Which country do you live in?
6 Q2 What is your gender?
7 Q3 What is the highest level of education you've attained?
8 Q4 What region of the UK do you live in?
9 Q5 What is your postcode? For example, if your postcode is 'SW1A 1AA', please enter 'SW1A 1' - the first part of the postcode and the first digit of the second part.
10 Q7 How did you vote in the 2016 EU Referendum?
11 Q8.0 Did you vote in the 2019 General Election?
12 Q8.1 If so, how did you vote in the 2019 General Election?
13 Q9 How did you vote in the 2017 General Election?
14 Q10 If an election were held tomorrow, which party, if any, would you vote for?
15 Q11 Overall, do you approve or disapprove of the Government’s performance?
16 Q12.1 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.

## Dealing with follow up questions

In [271]:
print('Unique values for Q 8:',
      raw_df['Q8.0 Did you vote in the 2019 General Election?'].unique())
print('Unique values for Q 8.1 based on a "no" for Q 8:',
      raw_df[raw_df[raw_df.columns[11]] == 'No, I do not vote'].iloc[:,12].unique())

Unique values for Q 8: ['Yes, I voted' nan 'No, I do not vote']
Unique values for Q 8.1 based on a "no" for Q 8: [nan]


In [272]:
raw_df[raw_df[raw_df.columns[11]] == 'No, I do not vote'].iloc[:, 11:13]

,Q8.0 Did you vote in the 2019 General Election?,"Q8.1 If so, how did you vote in the 2019 General Election?"
47,"No, I do not vote",NaN
53,"No, I do not vote",NaN
74,"No, I do not vote",NaN
78,"No, I do not vote",NaN
170,"No, I do not vote",NaN
184,"No, I do not vote",NaN
227,"No, I do not vote",NaN
245,"No, I do not vote",NaN
264,"No, I do not vote",NaN
280,"No, I do not vote",NaN


In [273]:
raw_df.loc[raw_df['Q8.0 Did you vote in the 2019 General Election?'] == 'No, I do not vote',
           'Q8.1 If so, how did you vote in the 2019 General Election?'] = "I didn't vote"

In [274]:
print('Unique values for Q 8:',
      raw_df['Q8.0 Did you vote in the 2019 General Election?'].unique())
print('Unique values for Q 8.1 based on a "no" for Q 8:',
      raw_df[raw_df[raw_df.columns[11]] == 'No, I do not vote'].iloc[:,12].unique())

Unique values for Q 8: ['Yes, I voted' nan 'No, I do not vote']
Unique values for Q 8.1 based on a "no" for Q 8: ["I didn't vote"]


In [309]:
raw_df[raw_df[raw_df.columns[11]] == 'No, I do not vote'].iloc[:, 11:13]

,Q8.0 Did you vote in the 2019 General Election?,"Q8.1 If so, how did you vote in the 2019 General Election?"
47,"No, I do not vote",I didn't vote
53,"No, I do not vote",I didn't vote
74,"No, I do not vote",I didn't vote
78,"No, I do not vote",I didn't vote
170,"No, I do not vote",I didn't vote
184,"No, I do not vote",I didn't vote
227,"No, I do not vote",I didn't vote
245,"No, I do not vote",I didn't vote
264,"No, I do not vote",I didn't vote
280,"No, I do not vote",I didn't vote


## Converting years to age

In [276]:
years = pd.to_numeric(raw_df['Q21 What is your year of birth?'], errors = 'coerce')

In [277]:

  
def calculateAge(year):
    try:
        today = date.today() 
        age = today.year -  year
    except ValueError:
        pass
    return age 
    

In [278]:
Respondant_Age = calculateAge(years)
raw_df['Q21 What is your year of birth?'] = Respondant_Age
raw_df.rename(columns = {'Q21 What is your year of birth?':'Respondant age'}, inplace = True) 
raw_df.head()

,is_preview,survey_id,respondent_id,start_time,end_time,Q1 Which country do you live in?,Q2 What is your gender?,Q3 What is the highest level of education you've attained?,Q4 What region of the UK do you live in?,"Q5 What is your postcode? For example, if your postcode is 'SW1A 1AA', please enter 'SW1A 1' - the first part of the postcode and the first digit of the second part.",...,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 7th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 8th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The NHS and social care,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The level of immigration,"Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Transport and infrastructure (eg. roads, trains, bus routes)",Q19,Q20 In which region of the UK do you reside?,Respondant age,Q22,Q23
0,False,7858f084-89fc-4d61-9659-5bec93ef7706,1596309010,2020-08-01 20:37:40,2020-08-01 20:47:24,England,Female,"CSE grade 1, GCE O level, GCSE, School Certifi...",South West,EX2 8,...,Education and schools,Protecting the environment / climate change,Ranked 5th,Ranked 4th,Ranked 6th,NaN,South West,80.0,NaN,NaN
1,False,NaN,1246553700,2020-08-03 19:43:51,2020-08-03 19:53:20,England,NaN,GCE A level or Higher Certificate,East of England,CM18 7,...,NaN,NaN,NaN,NaN,NaN,NaN,East of England,34.0,NaN,NaN
2,False,7858f084-89fc-4d61-9659-5bec93ef7706,9860644420,2020-07-30 18:52:26,2020-07-30 19:00:14,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",East Midlands,NG7 2,...,NaN,The level of immigration,Ranked 4th,Ranked 8th,np.nan,NaN,East Midlands,25.0,NaN,NaN
3,False,7858f084-89fc-4d61-9659-5bec93ef7706,3825252102,2020-08-03 05:47:35,2020-08-03 05:56:37,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",South East,BN3 1,...,Britain leaving the EU,The level of immigration,Ranked 3rd,Ranked 8th,NaN,NaN,South East,NaN,NaN,NaN
4,False,7858f084-89fc-4d61-9659-5bec93ef7706,4625262873,2020-07-31 14:24:37,2020-07-31 14:33:15,England,Female,City & Guilds certificate - advanced,North West,OL13 0,...,Protecting the environment / climate change,"Transport and infrastructure (eg. roads, train...",Ranked 5th,Ranked 4th,Ranked 8th,NaN,North West,45.0,NaN,NaN


## Mapping postcodes to westminster constituencies 

In [279]:
westminster_constituency = pd.merge(raw_df, postcode_lookup, 'left',
                                    left_on=raw_df.columns[9],
                                    right_on= 'postcode').iloc[:,-1]

In [280]:
raw_df["Q5 What is your postcode? For example, if your postcode is 'SW1A 1AA', please enter 'SW1A 1' - the first part of the postcode and the first digit of the second part."] = westminster_constituency
raw_df.rename(columns = {"Q5 What is your postcode? For example, if your postcode is 'SW1A 1AA', please enter 'SW1A 1' - the first part of the postcode and the first digit of the second part.":
                         "Respondant constituency"},
              inplace = True) 

raw_df.head()

,is_preview,survey_id,respondent_id,start_time,end_time,Q1 Which country do you live in?,Q2 What is your gender?,Q3 What is the highest level of education you've attained?,Q4 What region of the UK do you live in?,Respondant constituency,...,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 7th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 8th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The NHS and social care,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The level of immigration,"Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Transport and infrastructure (eg. roads, trains, bus routes)",Q19,Q20 In which region of the UK do you reside?,Respondant age,Q22,Q23
0,False,7858f084-89fc-4d61-9659-5bec93ef7706,1596309010,2020-08-01 20:37:40,2020-08-01 20:47:24,England,Female,"CSE grade 1, GCE O level, GCSE, School Certifi...",South West,E14000698,...,Education and schools,Protecting the environment / climate change,Ranked 5th,Ranked 4th,Ranked 6th,NaN,South West,80.0,NaN,NaN
1,False,NaN,1246553700,2020-08-03 19:43:51,2020-08-03 19:53:20,England,NaN,GCE A level or Higher Certificate,East of England,E14000729,...,NaN,NaN,NaN,NaN,NaN,NaN,East of England,34.0,NaN,NaN
2,False,7858f084-89fc-4d61-9659-5bec93ef7706,9860644420,2020-07-30 18:52:26,2020-07-30 19:00:14,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",East Midlands,E14000867,...,NaN,The level of immigration,Ranked 4th,Ranked 8th,np.nan,NaN,East Midlands,25.0,NaN,NaN
3,False,7858f084-89fc-4d61-9659-5bec93ef7706,3825252102,2020-08-03 05:47:35,2020-08-03 05:56:37,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",South East,E14000755,...,Britain leaving the EU,The level of immigration,Ranked 3rd,Ranked 8th,NaN,NaN,South East,NaN,NaN,NaN
4,False,7858f084-89fc-4d61-9659-5bec93ef7706,4625262873,2020-07-31 14:24:37,2020-07-31 14:33:15,England,Female,City & Guilds certificate - advanced,North West,E14000902,...,Protecting the environment / climate change,"Transport and infrastructure (eg. roads, train...",Ranked 5th,Ranked 4th,Ranked 8th,NaN,North West,45.0,NaN,NaN


## Removing useless columns

In [160]:
len(raw_df.columns)

72

In [161]:
no_useless_columns = raw_df[raw_df.columns[raw_df.count() > 10]]
len(no_useless_columns.columns)

69

In [162]:
no_useless_columns

,is_preview,survey_id,respondent_id,start_time,end_time,Q1 Which country do you live in?,Q2 What is your gender?,Q3 What is the highest level of education you've attained?,Q4 What region of the UK do you live in?,Respondant constituency,...,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 4th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 5th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 6th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 7th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 8th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The NHS and social care,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The level of immigration,"Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Transport and infrastructure (eg. roads, trains, bus routes)",Q20 In which region of the UK do you reside?,Respondant age
0,False,7858f084-89fc-4d61-9659-5bec93ef7706,1596309010,2020-08-01 20:37:40,2020-08-01 20:47:24,England,Female,"CSE grade 1, GCE O level, GCSE, School Certifi...",South West,E14000698,...,The level of immigration,The NHS and social care,"Transport and infrastructure (eg. roads, train...",Education and schools,Protecting the environment / climate change,Ranked 5th,Ranked 4th,Ranked 6th,South West,80.0
1,False,NaN,1246553700,2020-08-03 19:43:51,2020-08-03 19:53:20,England,NaN,GCE A level or Higher Certificate,East of England,E14000729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,East of England,34.0
2,False,7858f084-89fc-4d61-9659-5bec93ef7706,9860644420,2020-07-30 18:52:26,2020-07-30 19:00:14,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",East Midlands,E14000867,...,The NHS and social care,Education and schools,"Transport and infrastructure (eg. roads, train...",NaN,The level of immigration,Ranked 4th,Ranked 8th,np.nan,East Midlands,25.0
3,False,7858f084-89fc-4d61-9659-5bec93ef7706,3825252102,2020-08-03 05:47:35,2020-08-03 05:56:37,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",South East,E14000755,...,"Transport and infrastructure (eg. roads, train...",Protecting the environment / climate change,Helping your area recover from Covid-19,Britain leaving the EU,The level of immigration,Ranked 3rd,Ranked 8th,NaN,South East,NaN
4,False,7858f084-89fc-4d61-9659-5bec93ef7706,4625262873,2020-07-31 14:24:37,2020-07-31 14:33:15,England,Female,City & Guilds certificate - advanced,North West,E14000902,...,The level of immigration,The NHS and social care,Education and schools,Protecting the environment / climate change,"Transport and infrastructure (eg. roads, train...",Ranked 5th,Ranked 4th,Ranked 8th,North West,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,False,NaN,6682074886,2020-08-03 13:18:47,2020-08-03 13:26:42,England,Female,"University or CNAA higher degree (e.g. M.Sc, P...",East Midlands,E14000783,...,Jobs and the economy,Protecting the environment / climate change,Education and schools,"Transport and infrastructure (eg. roads, train...",The level of immigration,Ranked 2nd,Ranked 8th,Ranked 7th,East Midlands,64.0
890,NaN,7858f084-89fc-4d61-9659-5bec93ef7706,4489644514,2020-07-31 15:38:31,2020-07-31 15:46:24,England,Male,"University or CNAA higher degr

## Deleting duplicate repondants

In [163]:
len(raw_df)

894

In [164]:
no_duplicates = raw_df.drop_duplicates(subset='respondent_id')
len(no_duplicates)

815

In [165]:
d = raw_df.pivot_table(index=['respondent_id'], aggfunc='size')
print (d[d>1])

respondent_id
1066041204    2
1346571980    2
1692876912    2
1926771000    2
2025867713    2
2066257130    2
2269341929    2
2545471689    2
3126749573    2
3713348745    2
3846008545    2
4383795974    2
4848549806    2
6158925231    2
6620898156    2
6967692015    2
7331005066    2
7419903196    2
8253752319    2
8661542077    2
8702071915    2
8757564254    2
9295985745    2
936200890     2
dtype: int64


In [166]:
raw_df[raw_df.respondent_id == '4848549806']

,is_preview,survey_id,respondent_id,start_time,end_time,Q1 Which country do you live in?,Q2 What is your gender?,Q3 What is the highest level of education you've attained?,Q4 What region of the UK do you live in?,Respondant constituency,...,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 7th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 8th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The NHS and social care,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The level of immigration,"Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Transport and infrastructure (eg. roads, trains, bus routes)",Q19,Q20 In which region of the UK do you reside?,Respondant age,Q22,Q23
613,NaN,7858f084-89fc-4d61-9659-5bec93ef7706,4848549806,2020-07-30 01:46:35,2020-07-30 01:54:59,England,Female,"Other technical, professional or higher qualif...",South West,E14001000,...,The level of immigration,Protecting the environment / climate change,Ranked 2nd,Ranked 7th,Ranked 5th,NaN,South West,51.0,NaN,NaN
846,False,7858f084-89fc-4d61-9659-5bec93ef7706,4848549806,2020-07-30 22:38:46,2020-07-30 22:47:17,England,Female,"Other technical, professional or higher qualif...",South West,E14001000,...,The level of immigration,Protecting the environment / climate change,Ranked 2nd,Ranked 7th,Ranked 5th,NaN,South West,49.0,NaN,NaN


In [167]:
raw_df.iloc[424].values

array([False, '7858f084-89fc-4d61-9659-5bec93ef7706', '8702071915',
       '2020-08-01 15:54:02', '2020-08-01 16:04:00', 'England', 'Male',
       'University or CNAA higher degree (e.g. M.Sc, Ph.D) ',
       'Yorkshire and the Humber', nan, nan, 'Yes, I voted', 'Labour',
       'Labour', 'Labour', 'Strongly disapprove', '0-9', '81-90', '10-19',
       '0-9', '10-19', '20-29', '20-29', '0-9', '10-19', '0-9', '0-9',
       '0-9', '71-80', '71-80', '0-9', '50', 'No', 'Yes', 'No',
       'Ranked 3rd', 'Ranked 5th', 'Ranked 1st', 'Ranked 4th',
       'Ranked 8th', 'Helping the country recover from Covid-19',
       'The NHS and social care', 'Britain leaving the EU',
       'Jobs and the economy', 'Education and schools',
       'The level of immigration',
       'Transport and infrastructure (eg. roads, trains, bus routes)',
       'Protecting the environment / climate change', 'Ranked 2nd',
       'Ranked 6th', 'Ranked 7th', 'Ranked 4th', 'Ranked 5th',
       'Ranked 1st', 'Ranked 3rd', 

In [168]:
raw_df.iloc[851].values

array([False, '7858f084-89fc-4d61-9659-5bec93ef7706', '8702071915',
       '2020-08-02 21:26:57', '2020-08-02 21:36:38', 'England', 'Male',
       'University or CNAA higher degree (e.g. M.Sc, Ph.D) ',
       'Yorkshire and the Humber', nan, 'I voted to Remain',
       'Yes, I voted', 'Labour', 'Labour', 'Labour',
       'Strongly disapprove', '0-9', '81-90', '10-19', '0-9', '10-19',
       '20-29', '20-29', '0-9', '10-19', '0-9', '0-9', '0-9', '71-80',
       '71-80', '0-9', nan, 'No', 'Yes', 'No', 'Ranked 3rd', 'Ranked 5th',
       'Ranked 1st', 'Ranked 4th', 'Ranked 8th',
       'Helping the country recover from Covid-19', nan,
       'Britain leaving the EU', 'Jobs and the economy',
       'Education and schools', 'The level of immigration',
       'Transport and infrastructure (eg. roads, trains, bus routes)',
       'Protecting the environment / climate change', 'Ranked 2nd',
       'Ranked 6th', 'Ranked 7th', 'Ranked 4th', 'Ranked 5th',
       'Ranked 1st', 'Ranked 3rd', 'Ranked

## Dealing with Null values

In [169]:
raw_df['Q12.4 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Brexit Party'].unique()

array(['81-90', '50', 'None', '0-9', '71-80', '51-60', '20-29', '30-39',
       nan, '40-49', '10-19', '91-100', '61-70', 'np.nan', '-9999'],
      dtype=object)

In [173]:
nulls_replaced = raw_df.replace(['nan', '-9999', 'np.nan', 'None'], np.nan)

In [174]:
nulls_replaced['Q12.4 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Brexit Party'].unique()

array(['81-90', '50', nan, '0-9', '71-80', '51-60', '20-29', '30-39',
       '40-49', '10-19', '91-100', '61-70'], dtype=object)

## Tying it all together

In [397]:
def clean_survey_df(survey_df):
    
    # Dealing with follow up questions
    survey_df.loc[raw_df['Q8.0 Did you vote in the 2019 General Election?'] == 'No, I do not vote',
                  'Q8.1 If so, how did you vote in the 2019 General Election?'] = "I didn't vote"
    
    # Converting year of birth to age
    Respondant_Age = calculateAge(years)
    Respondant_Age = calculateAge(years)
    survey_df['Q21 What is your year of birth?'] = Respondant_Age
    survey_df.rename(columns = {'Q21 What is your year of birth?':'Respondant age'}, inplace = True) 
    
    # Converting post codes to westminster constituencies
    westminster_constituency = pd.merge(survey_df, postcode_lookup, 'left',
                                    left_on=survey_df.columns[9],
                                    right_on= 'postcode').iloc[:,-1]
    survey_df["Q5 What is your postcode? For example, if your postcode is 'SW1A 1AA', please enter 'SW1A 1' - the first part of the postcode and the first digit of the second part."] = westminster_constituency
    survey_df.rename(columns = {"Q5 What is your postcode? For example, if your postcode is 'SW1A 1AA', please enter 'SW1A 1' - the first part of the postcode and the first digit of the second part.":
                         "Respondant constituency"},
              inplace = True) 

    # Removing useless columns
    survey_df.drop(survey_df[survey_df.columns[survey_df.count() < 10]], axis = 1, inplace=True)
    
    # Deleting duplicates
    survey_df.drop_duplicates(subset='respondent_id', inplace=True)
    
    # Normalizing null values
    survey_df.replace(['nan', '-9999', 'np.nan', 'None'],
                      np.nan, inplace=True)
    
    # Resetting the index and returning the cleaned dataframe 
    clean_survey_df = survey_df.reset_index(drop=True)
    
    return clean_survey_df
    


In [398]:
raw = pd.read_csv(raw_data_fname)

In [399]:
clean_raw = clean_survey_df(raw)

In [400]:
clean_raw

,is_preview,survey_id,respondent_id,start_time,end_time,Q1 Which country do you live in?,Q2 What is your gender?,Q3 What is the highest level of education you've attained?,Q4 What region of the UK do you live in?,Respondant constituency,...,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 4th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 5th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 6th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 7th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 8th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The NHS and social care,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The level of immigration,"Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Transport and infrastructure (eg. roads, trains, bus routes)",Q20 In which region of the UK do you reside?,Respondant age
0,False,7858f084-89fc-4d61-9659-5bec93ef7706,1596309010,2020-08-01 20:37:40,2020-08-01 20:47:24,England,Female,"CSE grade 1, GCE O level, GCSE, School Certifi...",South West,E14000698,...,The level of immigration,The NHS and social care,"Transport and infrastructure (eg. roads, train...",Education and schools,Protecting the environment / climate change,Ranked 5th,Ranked 4th,Ranked 6th,South West,80.0
1,False,NaN,1246553700,2020-08-03 19:43:51,2020-08-03 19:53:20,England,NaN,GCE A level or Higher Certificate,East of England,E14000729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,East of England,34.0
2,False,7858f084-89fc-4d61-9659-5bec93ef7706,9860644420,2020-07-30 18:52:26,2020-07-30 19:00:14,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",East Midlands,E14000867,...,The NHS and social care,Education and schools,"Transport and infrastructure (eg. roads, train...",NaN,The level of immigration,Ranked 4th,Ranked 8th,NaN,East Midlands,25.0
3,False,7858f084-89fc-4d61-9659-5bec93ef7706,3825252102,2020-08-03 05:47:35,2020-08-03 05:56:37,England,Male,"University or CNAA higher degree (e.g. M.Sc, P...",South East,E14000755,...,"Transport and infrastructure (eg. roads, train...",Protecting the environment / climate change,Helping your area recover from Covid-19,Britain leaving the EU,The level of immigration,Ranked 3rd,Ranked 8th,NaN,South East,NaN
4,False,7858f084-89fc-4d61-9659-5bec93ef7706,4625262873,2020-07-31 14:24:37,2020-07-31 14:33:15,England,Female,City & Guilds certificate - advanced,North West,E14000902,...,The level of immigration,The NHS and social care,Education and schools,Protecting the environment / climate change,"Transport and infrastructure (eg. roads, train...",Ranked 5th,Ranked 4th,Ranked 8th,North West,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
810,False,7858f084-89fc-4d61-9659-5bec93ef7706,1564654256,2020-08-01 20:51:39,2020-08-01 21:01:24,England,Male,"CSE grade 1, GCE O level, GCSE, School Certifi...",Yorkshire and the Humber,E14000771,...,Jobs and the economy,Education and schools,"Transport and infrastructure (eg. roads, train...",Britain leaving the EU,NaN,Ranked 3rd,Ranked 2nd,Ranked 6th,Yorkshire and the Humber,56.0
811,False,NaN,6682074886,2020-08-03 13:18:47,2020-08-03 13:26:42,England,Female,"University or CNAA higher degree (e.g. M.Sc, P...",

## Tagging bad respondents

Signs of bad responses:
- fast survey completion times - DONE
- silly ages/postcodes/ages don't line up with event dates (i.e. saying you voted when too young) - DONE
- silly responses for freetext questions (age) - DONE
- many repeated answers on multiple choice questions
- Respondants didn't answer the questions (i.e. many null values across rows) - DONE

Trial on 1 row

In [402]:
trial = clean_raw[:1]

In [403]:
trial

,is_preview,survey_id,respondent_id,start_time,end_time,Q1 Which country do you live in?,Q2 What is your gender?,Q3 What is the highest level of education you've attained?,Q4 What region of the UK do you live in?,Respondant constituency,...,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 4th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 5th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 6th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 7th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Ranked 8th,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The NHS and social care,Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... The level of immigration,"Q18 Please rank the following issues in order of importance for *your local area* - with 1 as the most important and 8 as the least important.... Transport and infrastructure (eg. roads, trains, bus routes)",Q20 In which region of the UK do you reside?,Respondant age
0,False,7858f084-89fc-4d61-9659-5bec93ef7706,1596309010,2020-08-01 20:37:40,2020-08-01 20:47:24,England,Female,"CSE grade 1, GCE O level, GCSE, School Certifi...",South West,E14000698,...,The level of immigration,The NHS and social care,"Transport and infrastructure (eg. roads, train...",Education and schools,Protecting the environment / climate change,Ranked 5th,Ranked 4th,Ranked 6th,South West,80.0


### Checking survey length times

In [404]:
trial_survey_time = (trial.end_time.apply(pd.to_datetime) -
                     trial.start_time.apply(pd.to_datetime)) / pd.Timedelta(minutes=1)
trial_survey_time

0    9.733333
dtype: float64

In [405]:
clean_raw['survey time'] = (clean_raw.end_time.apply(pd.to_datetime) - 
                         clean_raw.start_time.apply(pd.to_datetime)) / pd.Timedelta(minutes=1)

In [406]:
clean_raw['survey time'].describe()

count    815.000000
mean       8.234274
std        1.540371
min        0.016667
25%        7.583333
50%        8.483333
75%        9.266667
max       10.000000
Name: survey time, dtype: float64

In [407]:
clean_raw['survey time'][clean_raw['survey time'] < 5] # 5 ~ 2*std from mean

36     0.300000
139    4.416667
149    4.566667
153    2.433333
245    2.600000
258    0.716667
261    3.316667
298    3.016667
318    3.233333
385    0.583333
397    3.466667
435    4.783333
470    0.016667
507    0.366667
525    0.733333
542    3.400000
549    0.983333
560    4.033333
577    2.483333
579    1.700000
588    4.183333
609    2.683333
626    0.733333
637    1.483333
643    3.350000
681    4.800000
700    2.033333
728    2.816667
742    0.350000
747    0.800000
762    2.866667
775    1.150000
814    2.916667
Name: survey time, dtype: float64

### Checking survey ages

In [408]:
clean_raw['Respondant age'].describe()

count    749.000000
mean      48.678238
std       17.689594
min        1.000000
25%       34.000000
50%       49.000000
75%       65.000000
max       84.000000
Name: Respondant age, dtype: float64

In [409]:
clean_raw['Respondant age'][clean_raw['Respondant age'] < 18] # 5 ~ 2*std from mean

210    1.0
325    1.0
508    1.0
667    1.0
Name: Respondant age, dtype: float64

### Checking for null values across rows

In [410]:
len(clean_raw[clean_raw.apply(lambda x: x.count(), axis=1) < 48])

65

In [411]:
len(clean_raw[clean_raw.isnull().sum(axis=1) > 20])

65

In [412]:
 clean_raw.iloc[:,13].unique()

array(['Conservative', "Don't know", "Didn't vote", 'Liberal Democrat',
       'Labour', nan, 'UK Independence Party (UKIP)', 'Green', 'Other'],
      dtype=object)

### Checking for age and voting inconsistencies

In [436]:
clean_raw.loc[(clean_raw.iloc[:,10] != "I didn't vote") & (clean_raw['Respondant age'] <= 19)].iloc[:,[10,11,12,13, -3]]

,Q7 How did you vote in the 2016 EU Referendum?,Q8.0 Did you vote in the 2019 General Election?,"Q8.1 If so, how did you vote in the 2019 General Election?",Q9 How did you vote in the 2017 General Election?,Respondant age
32,I voted to Remain,"Yes, I voted",Green,UK Independence Party (UKIP),18.0
106,I voted to Remain,"Yes, I voted",Labour,Labour,18.0
183,I voted to Leave,"Yes, I voted",Conservative,NaN,18.0
201,NaN,"Yes, I voted",I didn't vote,Didn't vote,18.0
203,I voted to Remain,"Yes, I voted",Labour,Labour,18.0
210,I voted to Remain,"No, I do not vote",I didn't vote,NaN,1.0
325,I voted to Leave,"Yes, I voted",Labour,Didn't vote,1.0
335,NaN,"Yes, I voted",I didn't vote,Didn't vote,19.0
375,I voted to Remain,"Yes, I voted",Labour,Labour,18.0
437,I voted to Remain,"Yes, I voted",Don't know,Don't know,19.0


In [448]:
clean_raw.iloc[:,10:14].apply(lambda col: col.str.contains(r'\bvote\b', na=False), axis=1)


Q7 How did you vote in the 2016 EU Referendum?                 94
Q8.0 Did you vote in the 2019 General Election?                26
Q8.1 If so, how did you vote in the 2019 General Election?    135
Q9 How did you vote in the 2017 General Election?             100
dtype: int64

In [445]:
clean_raw.iloc[:,13].str.contains("Didn't vote", regex=False).sum()

100

In [452]:
clean_raw.iloc[:,10].unique()

array(['I voted to Leave', 'I voted to Remain', "I didn't vote", nan,
       'Not sure'], dtype=object)

In [414]:
for i,col in enumerate(clean_raw.columns):
    print(i,col)

0 is_preview
1 survey_id
2 respondent_id
3 start_time
4 end_time
5 Q1 Which country do you live in?
6 Q2 What is your gender?
7 Q3 What is the highest level of education you've attained?
8 Q4 What region of the UK do you live in?
9 Respondant constituency
10 Q7 How did you vote in the 2016 EU Referendum?
11 Q8.0 Did you vote in the 2019 General Election?
12 Q8.1 If so, how did you vote in the 2019 General Election?
13 Q9 How did you vote in the 2017 General Election?
14 Q10 If an election were held tomorrow, which party, if any, would you vote for?
15 Q11 Overall, do you approve or disapprove of the Government’s performance?
16 Q12.1 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Conservatives
17 Q12.2 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards the

In [415]:
Q12_group = clean_raw.columns[16:23]
Q13_group = clean_raw.columns[23:32]
Q17_group = clean_raw.columns[35:51]
Q18_group = clean_raw.columns[51:67]

In [505]:
clean_raw[Q12_group].iloc[:,:-1]

,"Q12.1 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Conservatives","Q12.2 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Labour","Q12.3 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Liberal Democrats","Q12.4 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Brexit Party","Q12.5 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Greens","Q12.6 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Scottish National Party"
0,71-80,0-9,0-9,81-90,0-9,0-9
1,50,50,50,50,50,50
2,0-9,10-19,71-80,NaN,10-19,0-9
3,10-19,61-70,NaN,0-9,20-29,50
4,91-100,50,NaN,0-9,50,0-9
...,...,...,...,...,...,...
810,0-9,20-29,50,0-9,51-60,0-9
811,0-9,81-90,30-39,0-9,61-70,50
812,30-39,50,50,51-60,50,50
813,NaN,61-70,50,0-9,30-39,40-49


In [507]:
clean_raw[Q12_group].isin(clean_raw[Q12_group])

,"Q12.1 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Conservatives","Q12.2 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Labour","Q12.3 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Liberal Democrats","Q12.4 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Brexit Party","Q12.5 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Greens","Q12.6 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Scottish National Party","Q12.7 Below is a list of political parties. For each could you say on a scale of 0 to 100 how warmly you feel towards them with 0 meaning completely cold, 100 meaning completely warm and 50 meaning somewhere in the middle.... Plaid Cymru"
0,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True
2,True,True,True,False,True,True,True
3,True,True,False,True,True,True,True
4,True,True,False,True,True,True,True
...,...,...,...,...,...,...,...
810,True,True,True,True,True,True,True
811,True,True,True,True,True,True,True
812,True,True,True,True,True,True,True
813,False,True,True,True,True,True,True


In [417]:
# clean_raw['Extra Columns'] = clean_raw.where(clean_raw['survey time'] < 5)

In [475]:
clean_raw['Extra Column'] = np.where(clean_raw['survey time'] < 5, 'is_bad_respondant', np.nan)
len(clean_raw[clean_raw['Extra Column'] == 'is_bad_respondant'])

33

In [476]:
clean_raw.loc[clean_raw['Respondant age'] < 18, 'Extra Column'] = 'is_bad_respondant'
len(clean_raw[clean_raw['Extra Column'] == 'is_bad_respondant'])

37

In [477]:
clean_raw.loc[clean_raw.isnull().sum(axis=1) > 20, 'Extra Column'] = 'is_bad_respondant'
len(clean_raw[clean_raw['Extra Column'] == 'is_bad_respondant'])

99

In [478]:
clean_raw.iloc[:,10:14].apply(lambda col: ~col.str.contains(r'\bvote\b', na=False), axis=1).sum()

Q7 How did you vote in the 2016 EU Referendum?                721
Q8.0 Did you vote in the 2019 General Election?               789
Q8.1 If so, how did you vote in the 2019 General Election?    680
Q9 How did you vote in the 2017 General Election?             715
dtype: int64

In [479]:
clean_raw.iloc[:,10:14].apply(lambda col: ~col.str.contains(r'\bvote\b', na=False), axis=1)

,Q7 How did you vote in the 2016 EU Referendum?,Q8.0 Did you vote in the 2019 General Election?,"Q8.1 If so, how did you vote in the 2019 General Election?",Q9 How did you vote in the 2017 General Election?
0,True,True,True,True
1,True,True,True,True
2,False,True,False,False
3,True,True,True,True
4,True,True,True,True
...,...,...,...,...
810,True,True,True,True
811,True,True,True,True
812,True,True,True,True
813,True,True,True,True


In [480]:
clean_raw.loc[~clean_raw.iloc[:,10].str.contains(r'\bvote\b', na=False) &
          (clean_raw['Respondant age'] < 22),'Extra Column'] = 'is_bad_respondant'

len(clean_raw[clean_raw['Extra Column'] == 'is_bad_respondant'])

120

In [481]:
clean_raw.loc[~clean_raw.iloc[:,13].str.contains(r'\bvote\b', na=False) &
          (clean_raw['Respondant age'] < 21),'Extra Column'] = 'is_bad_respondant'

len(clean_raw[clean_raw['Extra Column'] == 'is_bad_respondant'])

122

In [498]:
condition_1 = clean_raw['Respondant age'] < 5
condition_2 = clean_raw.isnull().sum(axis=1) > 20
condition_3 = ~clean_raw.iloc[:,10].str.contains(r'\bvote\b',
                                                 na=False) & (clean_raw['Respondant age'] < 22)
condition_4 = ~clean_raw.iloc[:,13].str.contains(r'\bvote\b',
                                                 na=False) & (clean_raw['Respondant age'] < 21)
condition_5 = ~clean_raw.iloc[:,11].str.contains(r'\bvote\b',
                                                 na=False) & (clean_raw['Respondant age'] < 19)
condition_6 = ~clean_raw.iloc[:,12].str.contains(r'\bvote\b',
                                                 na=False) & (clean_raw['Respondant age'] < 19)

condition_7 = clean_raw['survey time'] < 5

In [499]:
clean_raw.loc[condition_1|condition_2|condition_3|condition_4
              |condition_5|condition_6|condition_7, 'bad_respondant'] = 'yes'

In [500]:
len(clean_raw[clean_raw['bad_respondant'] == 'yes'])

124

In [508]:
import sys

In [511]:
sys.path.append(r'C:\Users\bless\OneDrive\Documents\focaldatajuniorapplieddatascientist')

In [514]:
from final_sub import clean_survey_df

In [515]:
clean_survey_df

<function final_sub.clean_survey_df(survey_df)>